In [2]:
ls=sc.textFile('data/coupon150720.csv')

In [3]:
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [4]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [5]:
ls.count()

1232662

In [6]:
ls.cache()

MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:-2

In [7]:
ls.count()

1232662

In [8]:
ls.getNumPartitions()

3

In [9]:
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [ ]:
#cada linea es un pasajero con un codigo (0), nº de cupón (1), origen (2), destino (3), aerolinea vende (4), aerolinea opera, (5)

In [21]:
aerolineas = ls.map(lambda x: x.split(",")[5])

In [22]:
aerolineas.take(5)

[u'9W', u'9W', u'9W', u'9W', u'9W']

In [23]:
aerolineas.filter(lambda x: x== "IB")

PythonRDD[14] at RDD at PythonRDD.scala:43

In [24]:
aerolineas.filter(lambda x: x== "IB").count()

26158

In [25]:
ls.map(lambda x: x.split(",")[5]).filter(lambda x: x == "IB").count()

26158

In [ ]:
#hay 26.158 pasajeros volando por iberia

In [ ]:
#cuanto dinero estan haciendo estos pasajeros?

In [26]:
def getAerolineaDinero(l):
    elems = l.split(",")
    aerolinea = elems[5]
    dinero = float(elems[6])
    return (aerolinea, dinero)    
    

In [27]:
getAerolineaDinero(ls.first())

(u'9W', 56.79)

In [28]:
dineros = ls.map(getAerolineaDinero)

In [29]:
dineros.take(3)

[(u'9W', 56.79), (u'9W', 84.34), (u'9W', 60.0)]

In [32]:
dinerosIB = dineros.filter(lambda x: x[0] == "IB")

In [33]:
dinerosIB.map(lambda x: x[1]).sum()

2828044.4199999194

In [40]:
cantidades = dinerosIB.map(lambda x: x[1])

In [41]:
cantidades.stats()

(count: 26158, mean: 108.113939139, stdev: 168.011485542, max: 5239.93, min: 0.0)

In [42]:
cantidades.sum()

2828044.4199999194

In [ ]:
#respuesta: cantidades.sum()

In [43]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [45]:
def getRutaDinero(l):
    elems = l.split(",")
    origen = elems[2]
    destino = elems[3]
    dinero = float(elems[6])
    ruta = origen + "-" + destino
    return (ruta, dinero)

In [46]:
getRutaDinero(ls.first())

(u'MAA-AUH', 56.79)

In [ ]:
#top 5 con mas dinero

In [48]:
rutas = ls.map(getRutaDinero)

In [50]:
rutas.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [52]:
rutas.sortBy(lambda x: x[1], False).take(5)

[(u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'CDG-IAD', 637313.52),
 (u'IAD-CDG', 637313.52)]

In [ ]:
#el false es no ordenar descendiente

In [53]:
rutas.filter(lambda x: x[1] < 7500).sortBy(lambda x: x[1],False).take(5) 

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0)]

In [54]:
filtrado = rutas.filter(lambda x: x[1] < 7500)

In [55]:
filtrado.cache()

PythonRDD[51] at RDD at PythonRDD.scala:43

In [56]:
filtrado.groupByKey().take(3)

[(u'SZF-IST', <pyspark.resultiterable.ResultIterable at 0x7f8d6d8a6850>),
 (u'DME-RHO', <pyspark.resultiterable.ResultIterable at 0x7f8d6d8a6110>),
 (u'KRK-OSL', <pyspark.resultiterable.ResultIterable at 0x7f8d6d8a6810>)]

In [57]:
filtrado.groupByKey().mapValues(lambda vs: max(vs)).take(3)

[(u'SZF-IST', 289.83), (u'DME-RHO', 168.95), (u'KRK-OSL', 98.0)]

In [58]:
maximos = filtrado.groupByKey().mapValues(lambda vs:max(vs))

In [59]:
maximos.sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

In [ ]:
#el precio medio de la ruta

In [64]:
filtrado.groupByKey().mapValues(lambda vs: sum(vs)/len(vs)) \
.sortBy(lambda x: x[1], False) \
.take(5) \

[(u'BLA-MEC', 5164.355),
 (u'MEC-BLA', 5164.355),
 (u'CCS-MIA', 4350.516296296297),
 (u'MIA-CCS', 4186.95),
 (u'NRT-PPT', 3751.5499999999997)]

In [65]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [ ]:
#ruta aerolinea (4), dinero (6) y clase (9)

In [67]:
def getRutaALDineroClase(l):
    elems = l.split(",")
    ruta = elems[2] + "-" + elems[3]
    al = elems[4]
    dinero = float(elems[6])
    clase = elems[9]
    return ((ruta, al, clase), dinero)


In [68]:
getRutaALDineroClase(ls.first())

((u'MAA-AUH', u'9W', u'H'), 56.79)

In [69]:
import math

In [99]:
def valorLimite(vs):
    logs = [math.log(v+1) for v in vs]
    media = sum(logs)/len(logs)
    desv = [(x-media)**2 for x in logs]
    var = sum(desv)/len(desv)
    desvStd = math.sqrt(var)
    return math.exp(media + 2*desvStd) -1
    

In [100]:
valorLimite([12,3,4,1,2,4])

13.060852649090572

In [101]:
rutas = ls.map(getRutaALDineroClase)

In [102]:
rutas.take(5)

[((u'MAA-AUH', u'9W', u'H'), 56.79),
 ((u'AUH-CDG', u'9W', u'H'), 84.34),
 ((u'CJB-MAA', u'9W', u'H'), 60.0),
 ((u'DEL-DXB', u'9W', u'S'), 160.63),
 ((u'AUH-IXE', u'9W', u'V'), 152.46)]

In [103]:
rutas.groupByKey().take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7f8d6d982290>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7f8d6d982310>)]

In [104]:
def filtraCuponesExtremos(vs):
    vl = valorLimite(vs)
    f = [v for v in vs if v < vl]
    return f

In [105]:
grupos = rutas.groupByKey().mapValues(filtraCuponesExtremos)

In [107]:
def mediaGrupo(vs):
    if(len(vs) == 0):
        return 0.0
    else:
        return sum(vs)/len(vs)

In [108]:
grupos.mapValues(mediaGrupo).take(5)

[((u'TRD-BOO', u'SK', u'V'), 42.30263157894736),
 ((u'DPS-TMC', u'GA', u'N'), 60.7925),
 ((u'BEG-CDG', u'JU', u'K'), 0.0),
 ((u'SAV-CLT', u'US', u'N'), 0.0),
 ((u'DUS-AMS', u'KL', u'C'), 0.0)]

In [109]:
grupos.mapValues(mediaGrupo).sortBy(lambda x: x[1],False).take(5)

[((u'SYD-CGK', u'GA', u'G'), 6355194.0),
 ((u'COO-CDG', u'AF', u'J'), 538599.2),
 ((u'CDG-COO', u'AF', u'J'), 179628.3533333333),
 ((u'DKR-NBO', u'KQ', u'T'), 58253.593333333345),
 ((u'TSN-ICN', u'KE', u'Z'), 38655.45)]

In [110]:
grupos = rutas.groupByKey()

In [111]:
grupos.take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7f8d6d8555d0>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7f8d6d855a10>)]

In [115]:
gruposFiltrados = grupos.filter(lambda x: len(x[1]) > 10)

In [116]:
r = gruposFiltrados.mapValues(filtraCuponesExtremos).mapValues(mediaGrupo)

In [117]:
r.sortBy(lambda x: x[1], False).take(5)

[((u'MIA-CCS', u'S3', u'S'), 7826.25),
 ((u'CCS-MIA', u'S3', u'B'), 5550.0),
 ((u'CCS-MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'RUH-FRA', u'LH', u'F'), 4792.983),
 ((u'LHR-DOH', u'QR', u'F'), 4275.98923076923)]

In [118]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [119]:
ls.map(lambda x: x.split(",")[4]).distinct().take(5)

[u'', u'BE', u'WN', u'JV', u'WK']

In [120]:
ls.map(lambda x: x.split(",")[4]).distinct().count()

359

In [127]:
ls.map(lambda x: x.split(",")[7]).distinct().collect()

[u'USD']

In [128]:
ls.map(lambda x: x.split(",")[4]).distinct().saveAsTextFile("results.csv")